# BoZhon Robotic Kinematic Analysis
> Author: ZYL

In [20]:
# import
import roboticstoolbox as rtb
from roboticstoolbox.robot.DHLink import PrismaticDH, RevoluteDH
from spatialmath import *
import numpy as np
import math
import sympy as sp
import random
from error import *
# 改变后端，弹窗显示
%matplotlib qt5

In [21]:
# 标准的运动学模型
BZRobot = rtb.DHRobot([
    PrismaticDH(a=0, alpha=math.pi/2, theta=math.pi/2, offset=0),
    PrismaticDH(a=0, alpha=math.pi/2, theta=math.pi/2, offset=0),
    PrismaticDH(a=0, alpha=0, theta=0, offset=0),
    RevoluteDH(a=0, alpha=0, d=0),
], name="BoZhonRobot")
print(BZRobot.fkine([1, 1, 1, 0]).A)
BZRobot.plot([1, 1, 1, 0]);
print(BZRobot)

[[-6.1232340e-17  1.0000000e+00  1.2246468e-16  1.0000000e+00]
 [ 6.1232340e-17 -1.2246468e-16  1.0000000e+00  1.0000000e+00]
 [ 1.0000000e+00  6.1232340e-17 -6.1232340e-17  1.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
DHRobot: BoZhonRobot, 4 joints (PPPR), dynamics, standard DH parameters
┌──────┬─────┬────┬───────┐
│ θⱼ   │ dⱼ  │ aⱼ │  ⍺ⱼ   │
├──────┼─────┼────┼───────┤
│90.0° │  q1 │  0 │ 90.0° │
│90.0° │  q2 │  0 │ 90.0° │
│0.0°  │  q3 │  0 │  0.0° │
│ q4   │   0 │  0 │  0.0° │
└──────┴─────┴────┴───────┘

┌─┬──┐
└─┴──┘



In [22]:
# 旋转误差器，目前至考虑旋转带来的误差，所有的偏置均为0
# TODO：动画
delta = 1e-4
coeff = 1
Rotator = rtb.DHRobot([
    RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
    RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
    RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
], name="Rotator")
Rotator.plot([delta, delta, delta]);
R_error = Rotator.fkine([delta, delta, delta]).A
R_correct = Rotator.fkine([0, 0, 0]).A
print(R_correct)
print(R_error - R_correct)
# 这个结果完美符合理论公式

[[ 1.0000000e+00  1.2246468e-16 -1.2246468e-16  1.0000000e+00]
 [-1.2246468e-16  1.0000000e+00  1.2246468e-16  1.0000000e+00]
 [ 1.2246468e-16 -1.2246468e-16  1.0000000e+00  1.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
[[-1.00009999e-08 -9.99999993e-05  1.00009999e-04 -1.00004999e-04]
 [ 1.00009999e-04 -9.99999994e-09 -9.99899988e-05  9.99899998e-05]
 [-9.99999993e-05  9.99999998e-05 -9.99999994e-09  9.99999998e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]


In [14]:
# 平移误差器，目前仅考虑平移带来的误差
delta = 1e-4
Prismator = rtb.DHRobot([
    PrismaticDH(a=0, alpha=math.pi/2, offset=0*0.5, theta=math.pi/2),
    PrismaticDH(a=0, alpha=math.pi/2, offset=0*0.5, theta=math.pi/2),
    PrismaticDH(a=0, alpha=math.pi/2, offset=0*0.5, theta=math.pi/2),
], name="Prismator")
Prismator.plot([delta, delta, delta]);
Prismator.fkine([delta, delta, delta])
# 完美符合理论公式

   1         0         0         0.0001    
   0         1         0         0.0001    
   0         0         1         0.0001    
   0         0         0         1         


总结：
- 旋转误差器
~~~python
    RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),    # t_z
    RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),    # t_x
    RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),    # t_y
~~~
- 平移误差器
~~~python
    PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),   # dz
    PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),   # dx
    PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),   # dy
~~~

In [15]:
# 整体的运动学模型
coeff = 0
BZRobot = rtb.DHRobot([
    PrismaticDH(a=0, alpha=math.pi/2, theta=math.pi/2, offset=0), # LINK1
        # 平移误差器
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        # 旋转误差器
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
    PrismaticDH(a=0, alpha=math.pi/2, theta=math.pi/2, offset=0),   # LINK2
        # 平移误差器
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        # 旋转误差器
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
    PrismaticDH(a=0, alpha=0, theta=0, offset=0),   # LINK3
        # 平移误差器
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        # 旋转误差器
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
    RevoluteDH(a=0, alpha=0, d=0),  # LINK4
        # 平移误差器
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        PrismaticDH(a=0, alpha=math.pi/2, offset=coeff, theta=math.pi/2),
        # 旋转误差器
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
        RevoluteDH(a=0, alpha=math.pi/2, d=coeff, offset=math.pi/2),
], name="BZRobot")

BZRobot.plot([1]+[0]*6+[1]+[0]*6+[1]+[0]*6+[0]+[0]*6);
# TODO: 展示28个自由度

In [16]:
# Set value
L1_error = [d_z1, d_x1, d_y1, e_z1, e_x1, e_y1]
L2_error = [d_z2, d_x2, d_y2, e_z2, e_x2, e_y2]
L3_error = [d_z3, d_x3, d_y3, e_z3, e_x3, e_y3]
L4_error = [d_z4, d_x4, d_y4, t_z, e_x4, e_y4]  # 注意，e_z4已经被弃用
paraList = [p_z] + L1_error + [p_x] + L2_error + [p_y] + L3_error + [e_z4] + L4_error
len(paraList)

# NOTE!
# 注意：由于坐标系不同，所以误差的角标不加区分

# 位置误差控制在0.001m以内
displace_error = 0.001 * random.uniform(0, 1)
# 角度误差控制在1度，即0.017弧度以内
angle_error = 0.017 * 1 * 0.5 * random.uniform(0, 1)
# 末端为吸盘端，认为主要的角度误差产生在z轴上
angle_error_z = 0.017/2 * 0.5 * random.uniform(0, 1)
angle_error_x = 0.017/2 * 0.5 * random.uniform(0, 1)
angle_error_y = 0.017/2 * 0.5 * random.uniform(0, 1)

valueDic = {
    a1: 0, b1: 0, c1: 0,
    a2: 0, b2: 0, c2: 0,
    a3: 0, b3: 0, c3: 0,

    p_x: 0.1, p_y: 0.1, p_z: 0.1,
    
    e_x1: angle_error, e_y1: angle_error, e_z1: angle_error,
    e_x2: angle_error, e_y2: angle_error, e_z2: angle_error,
    e_x3: angle_error, e_y3: angle_error, e_z3: angle_error,
    e_x4: angle_error_x, e_y4: angle_error_y, 
    e_z4: 0,    # NOTE! e_z4已经被弃用!!! 设置为0！！！
    d_x1: displace_error, d_y1: displace_error, d_z1: displace_error,
    d_x2: displace_error, d_y2: displace_error, d_z2: displace_error,
    d_x3: displace_error, d_y3: displace_error, d_z3: displace_error,
    d_x4: displace_error, d_y4: displace_error, d_z4: displace_error,
    t_z: angle_error_z
}

subList = [
    (d_x1, valueDic[d_x1]), (d_y1, valueDic[d_y1]), (d_z1, valueDic[d_z1]),
    (d_x2, valueDic[d_x2]), (d_y2, valueDic[d_y2]), (d_z2, valueDic[d_z2]),
    (d_x3, valueDic[d_x3]), (d_y3, valueDic[d_y3]), (d_z3, valueDic[d_z3]),
    (d_x4, valueDic[d_x4]), (d_y4, valueDic[d_y4]), (d_z4, valueDic[d_z4]),
    (e_x1, valueDic[e_x1]), (e_y1, valueDic[e_y1]), (e_z1, valueDic[e_z1]),
    (e_x2, valueDic[e_x2]), (e_y2, valueDic[e_y2]), (e_z2, valueDic[e_z2]),
    (e_x3, valueDic[e_x3]), (e_y3, valueDic[e_y3]), (e_z3, valueDic[e_z3]),
    (e_x4, valueDic[e_x4]), (e_y4, valueDic[e_y4]), (e_z4, valueDic[e_z4]),
    (t_z, valueDic[t_z]),
    # 控制量
    (p_x, valueDic[p_x]), (p_y, valueDic[p_y]), (p_z, valueDic[p_z]),
    # 几何初始值（全部收缩在原点）
    (a1, 0), (b1, 0), (c1, 0),
    (a2, 0), (b2, 0), (c2, 0),
    (a3, 0), (b3, 0), (c3, 0),
]

In [17]:
expect_displacement = [valueDic[p_x]]+[0]*6+[valueDic[p_y]]+[0]*6+[valueDic[p_z]]+[0]*6+[0]+[0]*6
print(BZRobot.fkine(expect_displacement))   # 目标位置
mat_sim_exp = BZRobot.fkine(expect_displacement).A
BZRobot.plot(expect_displacement);
x_sim_exp = mat_sim_exp[0][3]
y_sim_exp = mat_sim_exp[1][3]
z_sim_exp = mat_sim_exp[2][3]
print(x_sim_exp, y_sim_exp, z_sim_exp)


   0         1         0         0.1       
   0         0         1         0.1       
   1         0         0         0.1       
   0         0         0         1         

0.10000000000000006 0.09999999999999998 0.09999999999999998


In [18]:
# 创造fkine对应的列表
real_displacement = []
for symbol in paraList:
    real_displacement.append(valueDic[symbol])

# 计算
mat_sim_real = BZRobot.fkine(real_displacement).A
print(mat_sim_real)
BZRobot.plot(real_displacement);
x_sim_real = mat_sim_real[0][3]
y_sim_real = mat_sim_real[1][3]
z_sim_real = mat_sim_real[2][3]
print(x_sim_real, y_sim_real, z_sim_real)

[[ 0.00383742  0.99997042 -0.00666511  0.10126259]
 [-0.00590976  0.00668772  0.99996017  0.10163881]
 [ 0.99997517 -0.00379787  0.00593525  0.10163992]
 [ 0.          0.          0.          1.        ]]
0.10126258592778878 0.10163881411425382 0.1016399220309783


In [19]:
precision = 5
# 仿真得到的误差
print("仿真得到的误差")
x_sim_error = x_sim_real - x_sim_exp
y_sim_error = y_sim_real - y_sim_exp
z_sim_error = z_sim_real - z_sim_exp
print(round(x_sim_error, precision), round(y_sim_error, precision), round(z_sim_error, precision))

# 理论计算的误差
P_err = P_err.subs(subList)
print("理论计算的误差")
x_theory_error = P_err[0]
y_theory_error = P_err[1]
z_theory_error = P_err[2]
print(round(x_theory_error, precision), round(y_theory_error, precision), round(z_theory_error, precision))

# 计算两者偏差
print("计算两者偏差")
x_bias_percent = (x_sim_error - x_theory_error) / x_theory_error
y_bias_percent = (y_sim_error - y_theory_error) / y_theory_error
z_bias_percent = (z_sim_error - z_theory_error) / z_theory_error
print(round(x_bias_percent, precision), round(y_bias_percent, precision), round(z_bias_percent, precision))

仿真得到的误差
0.00126 0.00164 0.00164
理论计算的误差
0.00172 0.00123 0.00159
计算两者偏差
-0.26747 0.33558 0.03115
